## car_lane_detection
### 2019-1 과제연구 
***


In [29]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
from moviepy.editor import VideoFileClip
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [30]:
def make_coordinates(image, line_parameters, rcnt, lcnt):
    slope, intercept = line_parameters
    #기울기   절편
    try:
        y1 = image.shape[0] #height
        y2 = int(y1*(3/5)) #height의 3/5까지 올라갈 때까지 탐색...
        x1 = int((y1 - intercept)/slope)
        x2 = int((y2 - intercept)/slope)
        return np.array([x1, y1, x2, y2]), rcnt, lcnt
    except OverflowError:
        return np.array([0, 0, 0, 0]), rcnt, lcnt

In [31]:
def average_slope_intercept(image, lines, rcnt, lcnt):
    left_fit = [] #왼쪽 line 의 평균
    right_fit = [] #오른쪽 line의 평균좌표
    try:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            parameters = np.polyfit((x1, x2), (y1, y2), 1)
            slope = parameters[0]
            intercept = parameters[1]

            if slope < -0.6: #line의 기울기가 음수인 경우
                left_fit.append((slope, intercept))
                #print('left slope', slope)
            if 18 > slope > 0.6: #line의 기울기가 양수인 경우
                right_fit.append((slope, intercept))
            if slope >= 18:
                print("can't detect!")
            cnt = 0 #cnt 초기화

    except TypeError:
        left_fit_average = []
        right_fit_average = []
    
    #위 모든 값들을 평균하여서 기울기와 y절편 계산하기
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    
    if left_fit_average != []:
        left_line, rcnt, lcnt = make_coordinates(image, left_fit_average, rcnt, lcnt)
    else: 
        left_line = np.array([0, 0, 0, 0])
    if right_fit_average != []:
        right_line, rcnt, lcnt = make_coordinates(image, right_fit_average, rcnt, lcnt)
    else:
        right_line = np.array([0, 0, 0, 0])
    if right_fit == []: rcnt += 1
    else: rcnt = 0
    if left_fit == []: lcnt += 1
    else: lcnt = 0
    #print(left_fit)
    return np.array([left_line, right_line]), rcnt, lcnt


In [32]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #Second. Gaussian Blur (to Reduce Noise)
    blur = cv2.GaussianBlur(gray, (5, 5), 0) #5x5 kernel size로 convolution 한다.
    #Apply Canny Method
    canny = cv2.Canny(blur, 10, 150) #급격한 변화만을 남겨 준다 (흰색으로 표시됨)
                                 #gradient 를 극단적으로 취해 준다
    return canny

In [33]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None: # ==not empty
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4) #2d array 를 1d array 로 변환
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

In [40]:
def region_of_interest(image):
    height = image.shape[0] #very simillar to 700
    width = image.shape[1]
    mwidth = width / 2
    polygons = np.array([
        [(int(width * 1 / 9), height), (int(width * 8 / 9), height), (int(mwidth), int(height * 3/9))]
    ])
    
    mask = np.zeros_like(image) #이미지와 같은 모양(pixel)으로 0배열을 만든다.
    cv2.fillPoly(mask, polygons, 255) #윤곽 만들기
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image


In [54]:
def ishide(image):
    avg = [0, 0, 0]
    s = image.shape
    width = s[1]
    height = s[0]

    for j in range(int(width)):
        for i in range(int(height*2/3), int(height)):
            bgr = image[int(i), int(j)]
            avg[0] += bgr[2]
            avg[1] += bgr[1]
            avg[2] += bgr[0]
    for i in range(3):
        avg[i] = avg[i] / (width * (height / 3))
    return avg

In [65]:
def amplify(image):
    s = image.shape
    #img = image.shape
    width = s[1]
    height = s[0]
    im = cv2.imread('pic.jpg')

    for j in range(int(width)):
        for i in range(int(height/2), int(height)):
            bgr = im[int(i), int(j)]
            if bgr[2] > 55 and bgr[2] < 120: im[int(i)][int(j)] = [255, 255, 255]
            #except IndexError: 
    return im

In [72]:
cap = cv2.VideoCapture("D:/#Datasets/2019-1/test_video_4.mp4")
rcnt = 0
lcnt = 0
color_avg_i = 0
while(cap.isOpened()):
    flag = True
    _, frame = cap.read()
    #save 'frame' to jpg file
    cv2.imwrite('pic.jpg', frame)
    frame_o = frame
    if color_avg_i == 0: color_avg_i = sum(ishide(frame))
    if abs(sum(ishide(frame)) - color_avg_i) >= 90: 
        frame = amplify(frame)
        print('wait...amplifying')
    
    #frame = cv2.imread('pic.jpg')
    #Firstly, convert image to grayscale
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image)
    
    #Hough Transform (Region of Interest) and draw lines
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]),
                            minLineLength = 40, maxLineGap = 5)
    averaged_lines, rcnt, lcnt = average_slope_intercept(frame, lines, rcnt, lcnt)
    if rcnt >= 5:
        if rcnt % 5 == 0: print('오른쪽 점선')
    if lcnt >= 5:
        if lcnt % 5 == 0: print('왼쪽 점선')
    #if lcnt >= 20:
    #    if lcnt % 5 == 0 : print('no line on left side')
    #if rcnt >= 20:
    #    if rcnt % 5 == 0: print('no line on right side')
    for item in averaged_lines:
        if max(item) > 10000: flag = False
    if flag == True : 
        line_image = display_lines(frame, averaged_lines)
        
    #averaged_lines를 통해서 line을 조금 더 부드럽게 만들기
    combo_image = cv2.addWeighted(frame_o, 0.8, line_image, 1, 1) 
                #가중치 부여해 이미지 합치기
    cv2.imshow("result", combo_image)
    if cv2.waitKey(1) == ord('q'): break #video 이므로 1ms씩 기다리기
        
cap.release()
cv2.destroyAllWindows()

In [122]:
import numpy as np
import cv2
img = cv2.imread('pic.jpg')
px = img[1, 1]
print(px)

[14 58 51]
